In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
processing = application_df.drop(columns=['EIN', 'NAME'])
processing.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
type_count = processing['APPLICATION_TYPE'].value_counts()
type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
replace_application = [x for x in processing['APPLICATION_TYPE'].value_counts().index if type_count[x] <= 500]
replace_application

for app in replace_application:
    processing.APPLICATION_TYPE = processing.APPLICATION_TYPE.replace(app,"Other")
processing.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
class_count = processing['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
replace_class = [x for x in processing['CLASSIFICATION'].value_counts().index if class_count[x] <= 1000]

# Replace in dataframe
for cls in replace_class:
    processing.CLASSIFICATION = processing.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
processing.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
cat_list = [x for x in processing.columns if processing[x].dtype == 'O']
cat_list
encoder = OneHotEncoder(sparse=False)

encode_df = pd.DataFrame(encoder.fit_transform(processing[cat_list]))

encode_df.columns = encoder.get_feature_names(cat_list)
application_df_processed = processing.merge(encode_df, right_index=True, left_index=True).drop(columns=cat_list)
application_df_processed

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34295,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34296,1,5000,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,1,5000,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
X = application_df_processed.drop(columns=['IS_SUCCESSFUL', 'SPECIAL_CONSIDERATIONS_N']).values
y = application_df_processed['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
def create_model(hp):
    nn_improved = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_improved.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_improved.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_improved.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_improved.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_improved

In [20]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [21]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 7,
 'num_layers': 2,
 'units_0': 3,
 'units_1': 5,
 'units_2': 5,
 'units_3': 5,
 'units_4': 5,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [ ]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [23]:
n_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 7
hidden_nodes_layer2 = 3
hidden_nodes_layer3 = 5

nn_improved = tf.keras.models.Sequential()
nn_improved.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=n_input_features, activation='sigmoid'))
nn_improved.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='sigmoid'))
nn_improved.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='sigmoid'))
nn_improved.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn_improved.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 7)                 301       
                                                                 
 dense_5 (Dense)             (None, 3)                 24        
                                                                 
 dense_6 (Dense)             (None, 5)                 20        
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________


In [28]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs('optimization_checkpoints/', exist_ok=True)
checkpoint_path = 'optimization_checkpoints/weights.{epoch:02d}.hdf5'

In [29]:
cp_callback = ModelCheckpoint(filepath=checkpoint_path, verbose=1,
                             save_weights_only=True, save_freq=5)

In [30]:
nn_improved.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_improved.fit(X_train_scaled, y_train, epochs=100, callbacks=cp_callback)

Epoch 1/500
  1/804 [..............................] - ETA: 6:53 - loss: 0.4996 - accuracy: 0.7812
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
  5/804 [..............................] - ETA: 14s - loss: 0.5717 - accuracy: 0.7375 
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
 10/804 [..............................] - ETA: 12s - loss: 0.5842 - accuracy: 0.7250
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
 15/804 [..............................] - ETA: 12s - loss: 0.5984 - accuracy: 0.7083
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
 20/804 [..............................] - ETA: 13s - loss: 0.5804 - accuracy: 0.7250
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
 25/804 [..............................] - ETA: 12s - loss: 0.5816 - accuracy: 0.7200
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
 34/804 [>.............................] - ETA: 10s - loss: 0.5802 - accur

285/804 [=========>....................] - ETA: 6s - loss: 0.5679 - accuracy: 0.7341
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
290/804 [=========>....................] - ETA: 6s - loss: 0.5684 - accuracy: 0.7338
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5

Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
300/804 [==========>...................] - ETA: 6s - loss: 0.5684 - accuracy: 0.7337
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
305/804 [==========>...................] - ETA: 6s - loss: 0.5691 - accuracy: 0.7329
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
310/804 [==========>...................] - ETA: 6s - loss: 0.5679 - accuracy: 0.7342
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
315/804 [==========>...................] - ETA: 6s - loss: 0.5687 - accuracy: 0.7337
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
320/804 [==========>.......

590/804 [=====================>........] - ETA: 2s - loss: 0.5740 - accuracy: 0.7274
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5

Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
600/804 [=====================>........] - ETA: 2s - loss: 0.5742 - accuracy: 0.7271
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
605/804 [=====================>........] - ETA: 2s - loss: 0.5741 - accuracy: 0.7273
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
611/804 [=====================>........] - ETA: 2s - loss: 0.5743 - accuracy: 0.7271
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5

Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
620/804 [======================>.......] - ETA: 2s - loss: 0.5746 - accuracy: 0.7270
Epoch 1: saving model to optimization_checkpoints\weights.01.hdf5
625/804 [======================>.......] - ETA: 2s - loss: 0.5747 - accuracy: 0.7269
Epoch 1: saving model to o

 87/804 [==>...........................] - ETA: 8s - loss: 0.5919 - accuracy: 0.7159
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
 91/804 [==>...........................] - ETA: 8s - loss: 0.5909 - accuracy: 0.7167
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
 96/804 [==>...........................] - ETA: 8s - loss: 0.5917 - accuracy: 0.7161
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
101/804 [==>...........................] - ETA: 8s - loss: 0.5932 - accuracy: 0.7141
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
106/804 [==>...........................] - ETA: 8s - loss: 0.5896 - accuracy: 0.7179
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
111/804 [===>..........................] - ETA: 8s - loss: 0.5894 - accuracy: 0.7182
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
116/804 [===>..........................] - ETA: 8s - loss: 0.5897 - accuracy: 0.7169
Epoch 2: 

371/804 [============>.................] - ETA: 5s - loss: 0.5764 - accuracy: 0.7276
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
378/804 [=============>................] - ETA: 5s - loss: 0.5761 - accuracy: 0.7279
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5

Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
386/804 [=============>................] - ETA: 5s - loss: 0.5765 - accuracy: 0.7274
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
391/804 [=============>................] - ETA: 5s - loss: 0.5763 - accuracy: 0.7275
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
396/804 [=============>................] - ETA: 5s - loss: 0.5759 - accuracy: 0.7282
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
401/804 [=============>................] - ETA: 5s - loss: 0.5759 - accuracy: 0.7282
Epoch 2: saving model to optimization_checkpoints\weights.02.hdf5
406/804 [==============>...

...

124/804 [===>..........................] - ETA: 7s - loss: 0.5445 - accuracy: 0.7366
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
129/804 [===>..........................] - ETA: 7s - loss: 0.5449 - accuracy: 0.7357
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
134/804 [====>.........................] - ETA: 7s - loss: 0.5447 - accuracy: 0.7358
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
139/804 [====>.........................] - ETA: 7s - loss: 0.5440 - accuracy: 0.7356
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
144/804 [====>.........................] - ETA: 7s - loss: 0.5468 - accuracy: 0.7342
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
151/804 [====>.........................] - ETA: 7s - loss: 0.5478 - accuracy: 0.7334
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
154/804 [====>.........................] - ETA: 7s - loss: 0.5479 - accuracy

394/804 [=============>................] - ETA: 4s - loss: 0.5449 - accuracy: 0.7345
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
399/804 [=============>................] - ETA: 4s - loss: 0.5455 - accuracy: 0.7340
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
404/804 [==============>...............] - ETA: 4s - loss: 0.5467 - accuracy: 0.7328
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5

Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
414/804 [==============>...............] - ETA: 4s - loss: 0.5455 - accuracy: 0.7335
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
419/804 [==============>...............] - ETA: 4s - loss: 0.5456 - accuracy: 0.7335
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
424/804 [==============>...............] - ETA: 4s - loss: 0.5458 - accuracy: 0.7332
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
433/80

689/804 [========================>.....] - ETA: 1s - loss: 0.5455 - accuracy: 0.7330
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
694/804 [========================>.....] - ETA: 1s - loss: 0.5458 - accuracy: 0.7328
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
699/804 [=========================>....] - ETA: 1s - loss: 0.5456 - accuracy: 0.7329
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
704/804 [=========================>....] - ETA: 1s - loss: 0.5457 - accuracy: 0.7329
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
709/804 [=========================>....] - ETA: 1s - loss: 0.5457 - accuracy: 0.7329
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
714/804 [=========================>....] - ETA: 1s - loss: 0.5451 - accuracy: 0.7335
Epoch 100: saving model to optimization_checkpoints\weights.100.hdf5
719/804 [=========================>....] - ETA: 0s - loss: 0.5446 - accuracy

160/804 [====>.........................] - ETA: 7s - loss: 0.5461 - accuracy: 0.7357
Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5
165/804 [=====>........................] - ETA: 7s - loss: 0.5446 - accuracy: 0.7369
Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5
170/804 [=====>........................] - ETA: 7s - loss: 0.5466 - accuracy: 0.7355
Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5
175/804 [=====>........................] - ETA: 7s - loss: 0.5479 - accuracy: 0.7350
Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5
180/804 [=====>........................] - ETA: 7s - loss: 0.5496 - accuracy: 0.7337
Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5

Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5
190/804 [======>.......................] - ETA: 7s - loss: 0.5499 - accuracy: 0.7326
Epoch 101: saving model to optimization_checkpoints\weights.101.hdf5

Epoch

KeyboardInterrupt: 

In [31]:
model_loss, model_accuracy = nn_improved.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5532 - accuracy: 0.7298 - 486ms/epoch - 2ms/step
Loss: 0.5532143115997314, Accuracy: 0.7297959327697754


In [32]:
nn_improved.save("AlphabetSoupCharity_Optimization.h5")